## Convert Classic Esri StoryMap Series (Tabbed Layout)
Fetch JSON from an ArcGIS Online hosted Classic Esri Story Map Series App and convert each tab into its own ArcGIS StoryMap with the cover supressed. Once converted, each ArcGIS StoryMap will open in a browser tab in order to complete the Story Checker. Once all are published, an ArcGIS StoryMap Collection is created that contains the converted app to replcate classic app look and feel.

TO DO - create conversion tool for classic swipe (second tab in Katrina story)

In [ ]:
# Check for required packages and install if necessary
import sys

def ensure_package(package, import_name=None):
    import_name = import_name or package
    try:
        __import__(import_name)
    except ImportError:
        try:
            print(f"Installing {package} ...")
            !{sys.executable} -m pip install {package}
        except Exception as e:
            print(f"Could not install {package}: {e}")

ensure_package("webcolors")
ensure_package("bs4")
ensure_package("arcgis")
ensure_package("Pillow", "PIL")
ensure_package("pandas")
ensure_package("requests")
ensure_package("threading")

TO DO - add cell describing keychain config

In [39]:
# Import packages, config, AGO authentication and helper functions
from bs4 import BeautifulSoup, NavigableString, Tag
from arcgis.apps.storymap import StoryMap, Themes, Image, Video, Audio, Embed, Map, Button, Text, Gallery, Timeline, Sidecar, Code, Table, TextStyles, Collection, CollectionNavigation
from arcgis.gis import GIS, Item
import arcgis.mapping
# try:
#     import arcgis.mapping
#     if "WebMap" in dir(arcgis.mapping):
#         from arcgis.mapping import WebMap as AGOLWebMap
#         print("WebMap is available in arcgis.mapping.")
#     else:
#         print("WebMap module not installed")
# except ImportError:
#     print("arcgis.mapping module is not installed")
from PIL import Image as PILImage
from io import BytesIO
from IPython.display import display
import pandas as pd
import webcolors
import webbrowser
import ipywidgets as widgets
from ipywidgets import IntProgress
import re, json, requests, sys, time, os, threading 

agoNotebook = False

# Set Pandas dataframe display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',1000)

# Print Python and ArcGIS for Python versions
# since things can change between versions
import sys
print(f"Python version: ",sys.version)
import arcgis
print("ArcGIS for Python API / StoryMap module version: ",arcgis.__version__)
# Connect to ArcGIS Online
# Define the GIS
if agoNotebook == False:
    import keyring
    service_name = "system" # Use the default local credential store
    success = False # Set initial state

    # Set a max number of attempts
    max_attempts = 5
    attempts = 0
    # Ask for the username
    while success == False and attempts < max_attempts:
        username_for_keyring = input("Enter your ArcGIS Online username:") # If you are using VS Code, the text input dialog box appears at the top of the window
        # Get the credential object
        credential = keyring.get_credential(service_name, username_for_keyring)
        # Check if the username is in the credential store
        if credential is None:
            print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
            attempts += 1
        # Retrieve the password, login and set the GIS portal
        else:
            password_from_keyring = keyring.get_password("system", username_for_keyring)
            portal_url = 'https://www.arcgis.com'  
            gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
            success = True
            # Print a success message with username and user's organization role
            print("Successfully logged in as: " + gis.properties.user.username, "(role: " + gis.properties.user.role + ")")
    if not success:
        print("Maximum login attempts reached. Exiting.")
        sys.exit(1)
else:
    gis = GIS("home")
    print("Successfully logged in as: " + gis.properties.user.username, "(role: " + gis.properties.user.role + ")")
    
# Helper functions

default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"

def animate_progress_bar(progress_bar, stop_event):
    while not stop_event.is_set():
        for pct in range(0, 101):
            if stop_event.is_set():
                break
            progress_bar.value = pct
            time.sleep(0.02)
        for pct in range(100, -1, -1):
            if stop_event.is_set():
                break
            progress_bar.value = pct
            time.sleep(0.02)

def color_to_hex(color_value):
    color_value = color_value.strip()
    # Check for rgb() format
    rgb_match = re.match(r'rgb-?(\d+)-?(\d+)-?(\d+)', color_value, re.IGNORECASE)
    if rgb_match:
        r, g, b = map(int, rgb_match.groups())
        return '{:02X}{:02X}{:02X}'.format(r, g, b)
    # Check for named color
    try:
        return webcolors.name_to_hex(color_value.lower())
    except ValueError:
        pass
    # Already hex
    if color_value.startswith('#') and len(color_value) == 7:
        return color_value.upper()
    return None

def convert_color_style_to_class(tag):
    # Check if tag has 'style' attribute with color
    style = tag.get('style', '')
    # Regex to find color property (hex, rgb, named colors)
    match = re.search(r'color\s*:\s*([^;]+)', style, re.IGNORECASE)
    if match:
        color_value = match.group(1).strip()
        # Convert hex (#XXXXXX) to class name, removing #
        if color_value.startswith('#'):
            class_color = f"sm-text-color-{color_value[1:].upper()}"
        else:
            # For rgb or named color, sanitize usable string (replace spaces/paren)
            sanitized = re.sub(r'[\s\(\)]', '', color_value).replace(',', '-')
            hex_color = color_to_hex(sanitized)
            class_color = f"sm-text-color-{hex_color}"
        # Remove color from style attribute
        new_style = re.sub(r'color\s*:\s*[^;]+;?', '', style, flags=re.IGNORECASE).strip()
        if new_style:
            tag['style'] = new_style
        else:
            del tag['style']
        # Add or append class attribute
        if 'class' in tag.attrs:
            tag['class'].append(class_color)
        else:
            tag['class'] = [class_color]

def process_html_colors_preserve_html(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    # Iterate over tags that can have styles: div, span, strong, em, p, etc.
    for tag in soup.find_all(True):
        convert_color_style_to_class(tag)
    return str(soup)

def convert_element_to_storymap_object(el):
    img_tag = el.find('img')
    if img_tag:
        src = img_tag.get("src")
        # Upgrade http to https if needed
        if src and src.startswith("http://"):
            src = "https://" + src[len("http://"):]
        alt = img_tag.get("alt", "")
        link = "" # TO DO handle occasions when image is intended to launch a link
        # Find figcaption in parent figure or div
        figcaption = ""
        # print("img_tag:", img_tag)
        parent_figure = img_tag.find_parent("figure")
        # print("parent_figure:", parent_figure)
        if parent_figure:
            caption_tag = parent_figure.find("figcaption")
            # print("caption_tag:", caption_tag)
            if caption_tag:
                figcaption = caption_tag.get_text(strip=True)
        else:
            # Try to find figcaption in the parent div
            parent_div = img_tag.find_parent("div")
            # print("parent_div:", parent_div)
            if parent_div:
                caption_tag = parent_div.find("figcaption")
                # print("caption_tag (div):", caption_tag)
                if caption_tag:
                    figcaption = caption_tag.get_text(strip=True)
        # print("Extracted figcaption:", figcaption, type(figcaption))
        img = Image(path=src)
        #img.link = link
        #img.image = src
        return img, figcaption, alt, link

    tag_name = el.name
    if tag_name == "p": # or tag_name in ["span", "strong", "em", "div"]:
        # Extract inner HTML preserving inline styles
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)

    elif tag_name == "video":
        src = el.get("src")
        alt = el.get("alt", "")
        vid = Video(path=src)
        vid.alt_text = alt
        vid.caption = "" # TO DO try to find Classic stories that have Videos with captions
        vid.video = src # Assign video property. TO DO fix this for hosted videos
        return vid
    
    elif tag_name == "audio":
        src = el.get("src")
        alt = el.get("alt", "")
        aud = Audio(path=src)
        aud.alt_text = alt
        aud.caption = "" # TO DO try to find Classic stories that have Audio with captions
        aud.audio = src # Assign Audio property. TO DO fix this for hosted videos
        return aud
    
    elif tag_name == "iframe" or tag_name == "embed":
        src = el.get("src") or el.get("data-src")
        alt = el.get("alt", "")
        if src:
            emb = Embed(path=src)
            emb.alt_text = alt
            emb.caption = "" # TO DO try to find Classic stories that have Embeds with captions
            emb.link = src
        return emb

    elif tag_name == "map":
        src = el.get("src")
        alt = el.get("alt", "")
        extent = "" #TO DO get extent
        layers = "" # TO DO get map layers
        mp = Map(item="")
        mp.alt_text = alt
        mp.caption = "" # TO DO try to find Classic stories that have Maps in Sidecar panel with captions
        mp.map = src
        mp.map_layers = layers 
        mp.set_viewpoint = extent
        return aud
    
    else:
        # Fallback for unsupported or unknown types - treat as text
        inner_html = ''.join(str(c) for c in el.contents)
        processed_html = process_html_colors_preserve_html(inner_html)
        return Text(text=processed_html, style=TextStyles.PARAGRAPH)

def parse_root_elements(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    html_elements = []
    for child in soup.contents:
        if not getattr(child, 'name', None):
            continue

        # Check if the parent itself is meaningful
        has_text = child.get_text(strip=True) != ""
        has_img = child.find('img') is not None
        has_video = child.find('video') is not None
        has_audio = child.find('audio') is not None
        has_iframe = child.find('iframe') is not None
        has_embed = child.find('embed') is not None
        has_map = child.find('map') is not None
        is_meaningful = has_text or has_img or has_video or has_audio or has_iframe or has_embed or has_map

        # Check for meaningful children
        meaningful_children = []
        for c in child.children:
            if not getattr(c, 'name', None):
                continue
            c_has_text = c.get_text(strip=True) != ""
            c_has_img = c.find('img') is not None
            c_has_video = c.find('video') is not None
            c_has_audio = c.find('audio') is not None
            c_has_iframe = c.find('iframe') is not None
            c_has_embed = c.find('embed') is not None
            c_has_map = c.find('map') is not None
            if c_has_text or c_has_img or c_has_video or c_has_audio or c_has_iframe or c_has_embed or c_has_map:
                meaningful_children.append(c)

        # If there are meaningful children, add them
        if meaningful_children:
            html_elements.extend(meaningful_children)
            # Optionally, if the parent is also meaningful and not just a container, add it too
            # If you want to avoid duplicates, only add children
            continue

        # If no meaningful children, but parent is meaningful, add parent
        if is_meaningful:
            html_elements.append(child)

    return html_elements


def parse_nested_elements(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    soup_list = [child for child in soup.contents if getattr(child, 'name', None)]
    html_elements = []
    for element in soup_list:
        for c in element:
            if getattr(c, 'name', None):
                html_elements.append(c)
    return html_elements

def convert_html_elements_to_storymap_node(html_elements):
    content_nodes = []
    image_metadata = []  # To store (img, caption, alt, link) tuples
    for el in html_elements:
        node = convert_element_to_storymap_object(el)
        if isinstance(node, tuple):
            img, caption, alt, link = node
            content_nodes.append(img)
            image_metadata.append((img, caption, alt, link))
        elif node:
            content_nodes.append(node)
    return content_nodes, image_metadata

def get_thumbnail_path(path):
    if path and isinstance(path, str) and os.path.isfile(path):
        return path
    return default_thumbnail_path

def download_thumbnail(webmap_item, thumbnail_path, gis=None):
    if webmap_item.thumbnail:
        url = f"{webmap_item._portal.resturl}content/items/{webmap_item.id}/info/{webmap_item.thumbnail}"
        token = gis._con.token if gis else None
        params = {'token': token} if token else {}
        response = requests.get(url, params=params)
        with open(thumbnail_path, 'wb') as f:
            f.write(response.content)
        return thumbnail_path
    else:
        print("No thumbnail available for this web map. Using default thumbnail.")
        return default_thumbnail_path

def ensure_local_thumbnail(thumbnail_path, local_filename="default_storymap_thumbnail.png"):
    # If it's already a local file, just return it
    if thumbnail_path and os.path.isfile(thumbnail_path):
        return thumbnail_path
    # Otherwise, download it
    response = requests.get(thumbnail_path)
    with open(local_filename, "wb") as f:
        f.write(response.content)
    return local_filename

# def create_webmap_thumbnail(webmap_id, thumbnail_path):
#     webmap_item = gis.content.get(webmap_id)
#     webmap = GIS.map(webmap_item)
#     # Export map as image (extent, size, etc. can be specified)
#     img_bytes = webmap.export_map(size=[800, 600])
#     img = Image.open(BytesIO(img_bytes))
#     img.thumbnail((200, 133))
#     img.save(thumbnail_path)

def create_image_thumbnail(image_url, thumbnail_path):
    response = requests.get(image_url)
    img = PILImage.open(BytesIO(response.content))
    img.thumbnail((800, 600))
    img.save(thumbnail_path)
    return thumbnail_path

Python version:  3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 12:58:53) [MSC v.1929 64 bit (AMD64)]
ArcGIS for Python API / StoryMap module version:  2.4.1.3
Successfully logged in as: dasbury_storymaps (role: org_admin)


In [41]:
def run_storymap_conversion(classic_storymap_id, agoNotebook=False):
    global classic_item, entries, published_storymap_items, thumbnails, thumbnail_paths,new_theme
    story_output_box.clear_output()
    try:
        print(f"Starting conversion for StoryMap ID: {classic_storymap_id}")
        story_output_box.clear_output()
        try:       
            agoNotebook = agoNotebook
            classic_storymap_id = classic_storymap_id  

            # Define the Classic StoryMap item id
            #classic_storymap_id = '597d573e58514bdbbeb53ba2179d2359' # Katrina +10
            #classic_storymap_id = '0cc3344dd34648de8ba9cc3ca29675da' # World of Cheese
            #classic_storymap_id = '34934c03445649cd9fcb422a2a7279c7' # Nepal 2015 Earthquake
            #classic_storymap_id = 'e718240a01374e18b5a7b867afe2a27c' # Goooooooal!
            #classic_storymap_id = '2e30152b96504eb796aee265d4ff0d94' # Thanksgiving Dinner
            #classic_storymap_id = '3d82a8da7319406fbbc6fb37e8a65756' # Wildfires 2018
            #classic_storymap_id = '1824b594690d43c3b7a2f05fdefe9e9f' # Whitenose Syndrome

            # Fetch the StoryMap Item from AGO
            classic_item = Item(gis=gis,itemid=classic_storymap_id)
            # Fetch the StoryMap data
            classic_data = Item.get_data(classic_item)
            if classic_data == {}:
                print("ERROR: StoryMap to be converted must be hosted on ArcGIS Online. Check your story's HTML source code to troubleshoot further")
            elif type(classic_data) == dict:
                classic_item_json = json.dumps(classic_data)
                classic_item_data = json.loads(classic_item_json)
            else:
                classic_item_data = json.loads(classic_data)
            
            # Extract story data
            classic_story_settings = classic_item_data["values"]["settings"]
            classic_story_theme = classic_story_settings["theme"]
            classic_story_title = classic_item_data["values"]["title"]
            classic_story_subtitle = classic_item_data["values"].get("subtitle", "")
            classic_story_data = classic_item_data["values"]["story"]
            
            # Extract tabs (entries list)
            entries = classic_story_data["entries"]
            
            # Fetch theme group
            classic_theme_group = classic_story_theme["colors"]["group"]
            if classic_theme_group == "dark":
                new_theme = Themes.OBSIDIAN
            elif classic_theme_group == "light":
                new_theme = Themes.SUMMIT
            else:
                new_theme = Themes.SUMMIT # Default to light theme
            
            created_storymaps = []
            published_storymap_items = []
            thumbnails = {}
            thumbnail_paths = {}
            description_html = {}
            nested_elements_df = {}
            root_elements_df = {}
            content_nodes = {}
            content_image_metadata = {}
            main_stage_types = {}
            main_stage_content = {}
            text_panels = {}
            
            start_index = 0  # Change to the index of the entry you want to start from (0-based)
            end_index = len(entries) - 1  # Change to the index of the entry you want to end at (0-based)
            for i in range(start_index, end_index + 1):
                entry = entries[i]
                # Fetch entry title
                entry_title = entry.get("title")

                if agoNotebook == True:
                # Show the progress bar immediately
                    with story_output_box:
                        story_progress_bar = IntProgress(min=0, max=100, description=f"")
                        output_label = widgets.Label(value=f"Converting '{entry_title}'")
                        output_Hbox = widgets.HBox([widgets.Box([output_label],layout=widgets.Layout(flex='1')), story_progress_bar],layout=widgets.Layout(width='50%'))
                        display(output_Hbox)
                        stop_event = threading.Event()
                        t = threading.Thread(target=animate_progress_bar, args=(story_progress_bar, stop_event))
                        t.start()
                else:
                    # Show the progress bar immediately
                    story_progress_bar = IntProgress(min=0, max=100, description=f"")
                    output_label = widgets.Label(value=f"Converting '{entry_title}'")
                    output_Hbox = widgets.HBox([widgets.Box([output_label],layout=widgets.Layout(flex='1')), story_progress_bar],layout=widgets.Layout(width='50%', justify_content='flex-end'))
                    display(output_Hbox)
                    stop_event = threading.Event()
                    t = threading.Thread(target=animate_progress_bar, args=(story_progress_bar, stop_event))
                    t.start()
            
                # Create a new StoryMap
                story = StoryMap()
                story.theme(new_theme)
            
                # Create Sidecar immersive section
                sidecar = Sidecar(style="docked-panel")
            
                # Add Sidecar to story
                story.add(sidecar)
            
                # Determine media content for main stage
                media_info = entry.get("media", {})
                media_type = media_info.get("type")
                main_stage_types[i] = media_type
            
                main_stage_content[i] = None
                invalid_webmap = False
                if media_type == "webmap":
                    webmap_id = media_info.get('webmap', {}).get('id')
                    if webmap_id:
                        try:
                            main_stage_content[i] = Map(webmap_id)
                            invalid_webmap = False
                            thumbnail_path = download_thumbnail(Item(gis=gis, itemid=webmap_id), f"webmap_thumbnail_{i}.png", gis)
                            if not os.path.isfile(thumbnail_path):
                                thumbnail_path = default_thumbnail_path
                            thumbnail_paths[i] = thumbnail_path
                            thumbnails[i] = Image(thumbnail_path)
                        except Exception as e:
                            print(f"Error processing webmap {webmap_id}: {e}. WebMap may have been deleted or is otherwise inaccessible.")
                            invalid_webmap = True
                elif media_type == "webpage":
                    webpage_url = media_info.get("webpage", {}).get("url")
                    if webpage_url:
                        main_stage_content[i] = Embed(webpage_url)
                elif media_type == "image":
                    image_url = media_info.get("image", {}).get("url")
                    if image_url:
                        main_stage_content[i] = Image(image_url)
                        thumbnail_path = create_image_thumbnail(image_url=image_url, thumbnail_path=f"image_thumbnail_{i}.png")
                        if not os.path.isfile(thumbnail_path):
                            thumbnail_path = default_thumbnail_path
                        thumbnail_paths[i] = thumbnail_path
                        thumbnails[i] = Image(thumbnail_path)

                # Ensure that a thumbnail is set
                if i not in thumbnail_paths or not thumbnail_paths[i] or not os.path.isfile(thumbnail_paths[i]):
                    thumbnail_paths[i] = default_thumbnail_path
                    thumbnails[i] = Image(default_thumbnail_path)

                # Fetch content from description (HTML)
                description_html[i] = entry.get("description", "")
            
                # Convert description HTML to pandas dataframes for inspection
                # Get the elements (choose one of the functions)
                nested_elements = parse_nested_elements(description_html[i])
                root_elements = parse_root_elements(description_html[i])
            
                # Build a DataFrame with tag name, text, and raw HTML
                nested_elements_df[i] = pd.DataFrame([{
                    'tag': el.name,
                    'text': el.get_text(strip=True),
                    'img_url': el.find('img')['src'] if el.find('img') else None,
                    'img_alt': el.find('img')['alt'] if el.find('img') and el.find('img').has_attr('alt') else None,
                    'img_caption': el.find('figcaption').get_text(strip=True) if el.find('figcaption') else None,
                    #'img_link': "", # TO DO handle occasions when image is intended to launch a link
                    'raw_html': str(el)
                } for el in nested_elements])
            
                root_elements_df[i] = pd.DataFrame([{
                    'tag': el.name,
                    'text': el.get_text(strip=True),
                    'img_url': el.find('img')['src'] if el.find('img') else None,
                    'img_alt': el.find('img')['alt'] if el.find('img') and el.find('img').has_attr('alt') else None,
                    'img_caption': el.find('figcaption').get_text(strip=True) if el.find('figcaption') else None,
                    #'img_link': "", # TO DO handle occasions when image is intended to launch a link
                    'raw_html': str(el)
                } for el in root_elements])
            
                # Convert description HTML to StoryMap content nodes
                content_nodes[i], content_image_metadata[i] = convert_html_elements_to_storymap_node(parse_root_elements(description_html[i]))
                nested_content_nodes, nested_image_metadata = convert_html_elements_to_storymap_node(parse_nested_elements(description_html[i]))
            
                # Create text panel from narrative nodes
                text_panels[i] = Text(content_nodes[i])
                #story.add(text_panel)
            
                # Add a slide to the sidecar with main media (no text panel yet)
                #sidecar.add_slide(contents=nested_content_nodes,media=main_stage_content[i])
                sidecar.add_slide(contents=content_nodes[i], media=main_stage_content[i])
            
                # Assign metadata to each image in contents
                for img, caption, alt, link in content_image_metadata[i]:
                    if caption:
                        img.caption = caption
                    if alt:
                        img.alt_text = alt
                    if link:
                        img.link = link
            
                # Set webmap properties. Map must be added to the story before setting viewpoint
                if not invalid_webmap:
                    if media_type == "webmap":
                        # Set the extent for the map stage
                        extent_json = media_info.get('webmap', {}).get('extent')
                        if extent_json:
                            main_stage_content[i].set_viewpoint(extent=extent_json)  # Extent dict per docs
                        # Set layer visibility 
                        old_layers = media_info.get('webmap', {}).get('layers', [])
                        if old_layers:
                            if hasattr(main_stage_content[i], "map_layers"):
                                for new_lyr in main_stage_content[i].map_layers:
                                    for old_lyr in old_layers:
                                        if new_lyr['id'] == old_lyr['id']:
                                            new_lyr['visible'] = old_lyr['visibility']
                        elif "operationalLayers" in media_info.get('webmap', {}):
                            old_layers = media_info.get('webmap', {}).get('operationalLayers', [])
                            if hasattr(main_stage_content[i], "map_layers"):
                                for new_lyr in main_stage_content[i].map_layers:
                                    for old_lyr in old_layers:
                                        if 'id' in new_lyr and 'id' in old_lyr and new_lyr['id'] == old_lyr['id']:
                                            new_lyr['visible'] = old_lyr['visibility']
            
                # Set Cover properties
                cover_properties = story.content_list[0]
                cover_properties.title = entry_title
                cover_properties.byline = ""
                cover_properties.date = "none"
                thumbnail_path = get_thumbnail_path(thumbnail_paths[i])
                # Set the cover media (thumbnail)
                cover_properties.media = thumbnails[i]
                #cover_properties.media = createThumbnail() # figure out a way to create a thumbnail from the first Sidecar media item
            
                # As the Cover class does not include a setting to hide the cover, we hide it by adding the 'config' key
                # to the Cover json
                for k,v in story.properties['nodes'].items():
                    if v['type'] == 'storycover':
                        v['config'] = {'isHidden': 'true'}
            
                # Save and publish storymap
                story_title = entry_title
                story.save(title=story_title, tags=["auto-created"], publish=True)
            
                # TO DO add an AGO relationship so if an attempt is made to delete story from My Content a warning is issued that the story
                # is included in a Collection (and give the name/id of the Collection(s) where it is referenced)
                
                created_storymaps.append(story)
                # Get the item object
                if hasattr(story, '_item'):
                    published_story_item = story._item
                    published_story_item.update(thumbnail=thumbnail_paths[i])
                    published_storymap_items.append(published_story_item)
                else:
                    print("Could not find item for story:", story.title)
                    continue
                
                # Open a browser to launch the Story Checker and fully publish the story
                story_url = "https://storymaps.arcgis.com/stories/"+ published_story_item.id
                if agoNotebook == True:
                    with story_output_box:
                        story_status_box = widgets.VBox([story_output_box, widgets.HTML(value=f"Created <strong>'{published_story_item.title}'</strong>. Click the link to check for errors and to complete publishing."), widgets.HTML(value=f"<a href='{story_url}' target='_blank'>{story_url}</a>")])
                        display(story_status_box)
                        #print(f"Click on the link to complete publishing and check for errors --> '{published_story_item.title}' {story_url}")
                else:
                    story_output_box.clear_output()
                    story_status_box = widgets.VBox([story_output_box, widgets.HTML(value=f"Created <strong>'{published_story_item.title}'</strong>. Default browser will open. Check the story for errors and complete publishing if necessary.")])
                    display(story_status_box)
                    #story_output_box.append_stdout(f"Opening: {published_story_item.title} ({story_url})\n")
                    webbrowser.open(story_url)

                # Stop the progress bar animation
                stop_event.set()
                t.join()
                story_progress_bar.value = 100  # Set to 100% when done
                story_progress_bar.bar_style = 'success'  # 'success' makes it green
                story_progress_bar.description = "Done"  # Update the label to "Done"

            if agoNotebook == True:
                with story_output_box:
                    print(f"\nSuccess! Created {len(created_storymaps)} StoryMaps. Be sure to click on each link before creating Collection.")
                    # Check to ensure each story was successfully published (AGO Item object exists and json.published exists?)
                    # Disable collection button until all stories are successfully published
                    display(widgets.VBox([collection_submit_btn, collection_output_box]))
            else:
                story_output_box.clear_output()
                collection_submit_btn.description_tooltip = "Disabled until all stories have been successfully created"
                story_success_box = widgets.VBox([story_output_box, widgets.HTML(value=f"Success! Created <strong>{len(created_storymaps)} StoryMaps</strong>. Be sure to check each story for errors <strong>before creating Collection</strong>."), collection_submit_btn, collection_output_box])
                display(story_success_box)
                    # Check to ensure each story was successfully published (AGO Item object exists and json.published exists?)
                    # Disable collection button until all stories are successfully published
                time.sleep(5)  # Pauses execution for 5 seconds
                    # Enable the collection button
                collection_submit_btn.disabled = False
                collection_submit_btn.style.button_color = None
                collection_submit_btn.description_tooltip = "Click to create Collection of all converted stories"

        except Exception as e:
            if agoNotebook == True:
                # Stop the animation thread
                stop_event.set()
                t.join()

                # Set progress bar to 100%, update label and color
                story_progress_bar.value = 100
                story_progress_bar.description = "Error!"
                story_progress_bar.bar_style = 'danger'  # 'danger' makes it red
                with story_output_box:
                    print(f"Error: {str(e)}")
            else:
                print(f"Error: {str(e)}")

        print("Conversion complete.")
    except Exception as e:
        print(f"Error in run_storymap_conversion: {e}")


def run_create_collection(agoNotebook=False):
    global classic_item, published_storymap_items, thumbnails, thumbnail_paths, new_theme
    collection_output_box.clear_output()
    # Display a progress bar
    if agoNotebook == True:
        with collection_output_box:
            collection_progress_bar = IntProgress(min=0, max=100, description=f"")
            output_label = widgets.Label(value=f"Creating Collection")
            output_Hbox = widgets.HBox([widgets.Box([output_label],layout=widgets.Layout(flex='1')), collection_progress_bar])
            display(output_Hbox)
            stop_event = threading.Event()
            t = threading.Thread(target=animate_progress_bar, args=(collection_progress_bar, stop_event))
            t.start()
    else:
        collection_progress_bar = IntProgress(min=0, max=100, description=f"")
        output_label = widgets.Label(value=f"Creating Collection")
        output_Hbox = widgets.HBox([widgets.Box([output_label],layout=widgets.Layout(flex='1')), collection_progress_bar],layout=widgets.Layout(width='50%'))
        display(output_Hbox)
        stop_event = threading.Event()
        t = threading.Thread(target=animate_progress_bar, args=(collection_progress_bar, stop_event))
        t.start()
    
    try:
        # Fetch the classic story's thumbnail
        classic_thumbnail_path = download_thumbnail(Item(gis=gis, itemid=classic_item.itemid), "classic_story_thumbnail.png", gis)
        print("classic_thumbnail_path type:", type(classic_thumbnail_path))
        classic_thumbnail = Image(path=classic_thumbnail_path) # if classic_thumbnail_path else None
        # If no thumbnail found use the default
        if not classic_thumbnail:
            classic_thumbnail = Image(default_thumbnail_path)
        
        # Create a Collection to hold the created StoryMaps
        collection = Collection()
        collection_title = classic_item.title
        for i, story in enumerate(published_storymap_items):
            local_thumbnail = ensure_local_thumbnail(thumbnail_paths[i], f"storymap_thumbnail_{i}.png")
            collection.add(item=story, title=story.title, thumbnail=local_thumbnail)
        # Assign the classic story's title to the Cover object of the Collection
        collection.content[0].title = collection_title
        # Clear the byline from the Cover (can be modified in the builder)
        collection.content[0].byline = ""
        # Apply the same theme to the collection as the stories
        collection.theme(new_theme)    
        # Set the Collection to used the Tabbed Navigation type
        collection.content[1].type = "tab"
        # Set the Collection thumbnail to be the same as the classic story
        collection.content[1].media = classic_thumbnail # this doesn't seem to be working at the moment. Likely overwritten by the builder on save
        
        # Save and publish the Collection
        collection.save(title=collection_title, tags=["auto-created"], publish=True)
        
        # Stop the progress bar animation
        stop_event.set()
        t.join()
        collection_progress_bar.value = 100  # Set to 100% when done
        collection_progress_bar.bar_style = 'success'  # 'success' makes it green
        collection_progress_bar.description = "Done"  # Update the label to "Done"

        # Get the item object
        published_collection_item = collection._item
        collection_url = "https://storymaps.arcgis.com/collections/"+ published_collection_item.id
        if agoNotebook == True:
            with collection_output_box:
                print(f"Click on the link to open your new Collection '{collection_title}' --> {collection_url}")
        else:
            collection_output_box.clear_output()
            collection_success_box = widgets.VBox([collection_output_box, widgets.HTML(value=f"<b style='color:green;'>Success! Created Collection '{collection_title}'.</b>")])
            display(collection_success_box)
            print(f"Opening Collection: {collection_title} ({collection_url})")
            webbrowser.open(collection_url)

        
        with collection_output_box:
            print(f"Successfuly created '{collection_title}'")    
    except Exception as e:
        if agoNotebook == True:
            with collection_output_box:
                print(f"Error: {str(e)}")
            # Stop the animation thread
            stop_event.set()
            t.join()

            # Set progress bar to 100%, update label and color
            collection_progress_bar.value = 100
            collection_progress_bar.description = "Error!"
            collection_progress_bar.bar_style = 'danger'  # 'danger' makes it red
        else:
            print(f"Error: {str(e)}")
            # Stop the animation thread
            stop_event.set()
            t.join()

            # Set progress bar to 100%, update label and color
            collection_progress_bar.value = 100
            collection_progress_bar.description = "Error!"
            collection_progress_bar.bar_style = 'danger'  # 'danger' makes it red

   

if agoNotebook == True:
    # Create widgets for user interface
    # Create checkbox as a variable
    input_param1 = widgets.Checkbox(value=False, description="Yes")
    user_line1 = widgets.HBox([widgets.Label(value="Are you running this within ArcGIS Online?"),input_param1])
    input_param2 = widgets.Text(value="597d573e58514bdbbeb53ba2179d2359", description="Story ID:")
    user_line2 = widgets.HBox([widgets.Label(value="Paste 32-digit Classic Esri Story Map id -->"), input_param2]) # TO DO add error checking logic and warning
    story_submit_btn = widgets.Button(description="Convert Story")
    collection_submit_btn = widgets.Button(description="Create Collection", description_tooltip="Disabled until all stories have been successfully created")
    # Before displaying the button, disable and style it
    collection_submit_btn.disabled = True
    collection_submit_btn.style.button_color = 'gold'  # yellow color
    story_output_box = widgets.Output()
    collection_output_box = widgets.Output()

    classic_item = None
    entries = []
    published_storymap_items = []
    thumbnails = {}
    new_theme = None

    def on_story_submit(btn):
        run_storymap_conversion(input_param2.value, agoNotebook=True)
    story_submit_btn.on_click(on_story_submit)
    display(widgets.VBox([user_line2, user_line1, story_submit_btn, story_output_box]))

    def on_collection_submit(btn):
        run_create_collection(agoNotebook=True)
    collection_submit_btn.on_click(on_collection_submit)

else:
    # --- VS Code debug/test entry point ---
    # Set your test StoryMap ID here
    test_storymap_id = "597d573e58514bdbbeb53ba2179d2359"
    story_output_box = widgets.Output()  # Dummy, for compatibility
    print("Running in VS Code debug mode. Starting conversion...")
    run_storymap_conversion(test_storymap_id, agoNotebook=False)
    print("Creating collection...")
    run_create_collection(agoNotebook=False)


    

Running in VS Code debug mode. Starting conversion...
Starting conversion for StoryMap ID: 597d573e58514bdbbeb53ba2179d2359


Conversion complete.
Creating collection...


classic_thumbnail_path type: <class 'str'>


Opening Collection: Katrina +10: A Decade of Change in New Orleans  (https://storymaps.arcgis.com/collections/a60584bf7da748dea6f31809b939545a)


In [36]:
run_create_collection(agoNotebook=False)

classic_thumbnail_path type: <class 'str'>
Error: File(https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png) not found.


In [38]:
from arcgis.apps.storymap import Image
classic_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"
img = Image(path=classic_thumbnail_path)